In [6]:
import json

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

In [7]:
get_current_weather(location="Madrid", unit="celsius")

'{"temperature": 25, "unit": "celsius"}'

### Add System Prompt for tool usage

In [8]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

# Remember to load the environment variables. You should have the Groq API Key in there
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

In [9]:
# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [10]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather", "arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


In [11]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [12]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

In [13]:
result = get_current_weather(**parsed_output["arguments"])

In [14]:
result

'{"temperature": 25, "unit": "celsius"}'

In [15]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [16]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25°C.'

# Scaling UP and Putting everything together



In [29]:
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent
import json

@tool
def get_current_weather(location: str, unit: str):
    """
    Get the current weather in a given location.

    Args:
        location (str): The city and state, e.g. Madrid, Barcelona
        unit (str): The unit. It can take two values; "celsius", "fahrenheit"

    Returns:
        JSON string of weather information.
    """
    if location == "Madrid":
        return json.dumps({"temperature": 25, "unit": unit})
    else:
        return json.dumps({"temperature": 58, "unit": unit})

if __name__ == '__main__':
    agent = ToolAgent(tools=[get_current_weather])
    response = agent.run("What's the current temperature in Madrid, in Celsius?")
    print(response)


Using Tool: get_current_weather

Tool call dict: 
{'name': 'get_current_weather', 'arguments': {'location': 'Madrid', 'unit': 'celsius'}, 'id': 1}

Tool result: 
{"temperature": 25, "unit": "celsius"}
The current temperature in Madrid is 25°C.


# Additional Use Case Example:

In [30]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. 
    Each story contains the title, URL, ascore, author, and time of submission. The data is fetched 
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    
    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Get the top story IDs
        top_story_ids = response.json()[:top_n]
        
        top_stories = []
        
        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()
            
            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })
        
        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [31]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'Ask HN: Who is hiring? (June 2025)', 'url': 'No URL available'},
 {'title': 'Conformance Checking at MongoDB: Testing That Our Code Matches Our TLA+ Specs',
  'url': 'https://www.mongodb.com/blog/post/engineering/conformance-checking-at-mongodb-testing-our-code-matches-our-tla-specs'},
 {'title': 'Snowflake to buy Crunchy Data for $250M',
  'url': 'https://www.wsj.com/articles/snowflake-to-buy-crunchy-data-for-250-million-233543ab'},
 {'title': 'Show HN: I build one absurd web project every month',
  'url': 'https://absurd.website'},
 {'title': 'Show HN: Kan.bn – An open-source alterative to Trello',
  'url': 'https://github.com/kanbn/kan'}]

In [32]:
hn_tool = tool(fetch_top_hacker_news_stories)

In [33]:
hn_tool.name

'fetch_top_hacker_news_stories'

In [34]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, ascore, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

In [35]:
tool_agent = ToolAgent(tools=[hn_tool])

In [36]:
output = tool_agent.run(user_msg="Tell me your name")

In [37]:
print(output)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


In [38]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "Ask HN: Who is hiring? (June 2025)", "url": "No URL available"}, {"title": "Conformance Checking at MongoDB: Testing That Our Code Matches Our TLA+ Specs", "url": "https://www.mongodb.com/blog/post/engineering/conformance-checking-at-mongodb-testing-our-code-matches-our-tla-specs"}, {"title": "Snowflake to buy Crunchy Data for $250M", "url": "https://www.wsj.com/articles/snowflake-to-buy-crunchy-data-for-250-million-233543ab"}, {"title": "Show HN: I build one absurd web project every month", "url": "https://absurd.website"}, {"title": "Show HN: Kan.bn \u2013 An open-source alterative to Trello", "url": "https://github.com/kanbn/kan"}]


In [39]:
print(output)

Based on the provided information, the top 5 Hacker News stories are:

1. **Ask HN: Who is hiring? (June 2025)** - No URL available
2. **Conformance Checking at MongoDB: Testing That Our Code Matches Our TLA+ Specs** - https://www.mongodb.com/blog/post/engineering/conformance-checking-at-mongodb-testing-our-code-matches-our-tla-specs
3. **Snowflake to buy Crunchy Data for $250M** - https://www.wsj.com/articles/snowflake-to-buy-crunchy-data-for-250-million-233543ab
4. **Show HN: I build one absurd web project every month** - https://absurd.website
5. **Show HN: Kan.bn – An open-source alternative to Trello** - https://github.com/kanbn/kan
